In [2]:
# assuming we want to run the programme overnight i.e. 8hrs

total_time = 8 * 60 * 60 # in seconds
time_for_5k = 60 # in seconds

real_5k_time = time_for_5k * 1.1 # add any overhead

total_forward_passes = total_time // time_for_5k

print(total_forward_passes)

480


In [9]:
import torch
import re
from collections import defaultdict
from torchvision.models import vit_b_32, ViT_B_32_Weights

model = vit_b_32(weights=ViT_B_32_Weights.IMAGENET1K_V1)

total_params = sum(p.numel() for p in model.parameters())
print(f"\nModel: ViT_B_32")
print(f"Total Parameters: {total_params:,}\n")

grouped = defaultdict(list)

# Normalize and group parameter names
for name, p in model.named_parameters():
    # Replace numeric indices with # using regex
    normalized = re.sub(r'encoder\.layers\.encoder_layer_\d+', 'encoder_layer_#', name)
    grouped[normalized].append((name, tuple(p.shape), p.numel()))

# Prepare summary
summary = []
for group, entries in grouped.items():
    count = len(entries)
    total = sum(e[2] for e in entries)
    shape = entries[0][1]
    per_param = entries[0][2]
    summary.append((group, count, shape, per_param, total))

# Sort by total parameters in descending order
summary.sort(key=lambda x: x[3], reverse=True)

# Print formatted table
print(f"{'Group':<60} {'Count':>5} {'Shape':>20} {'Each':>12} {'Total':>15}")
print("-" * 120)
for group, count, shape, per_param, total in summary:
    print(f"{group:<60} {count:5} {str(shape):>20} {per_param:12,} {total:15,}")



Model: ViT_B_32
Total Parameters: 88,224,232

Group                                                        Count                Shape         Each           Total
------------------------------------------------------------------------------------------------------------------------
conv_proj.weight                                                 1     (768, 3, 32, 32)    2,359,296       2,359,296
encoder_layer_#.mlp.0.weight                                    12          (3072, 768)    2,359,296      28,311,552
encoder_layer_#.mlp.3.weight                                    12          (768, 3072)    2,359,296      28,311,552
encoder_layer_#.self_attention.in_proj_weight                   12          (2304, 768)    1,769,472      21,233,664
heads.head.weight                                                1          (1000, 768)      768,000         768,000
encoder_layer_#.self_attention.out_proj.weight                  12           (768, 768)      589,824       7,077,888
encoder.pos_e